In [1]:
from scipy.io import loadmat
import cobra
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd

In [2]:
# see what the coupling constraints are
model = cobra.io.load_matlab_model('Matlab_Models/Diet/microbiota_model_diet_SRS011061.mat')
mat = loadmat('Matlab_Models/Diet/microbiota_model_diet_SRS011061.mat', simplify_cells=True)

# see what the coupling constraints are
ref_model = cobra.io.load_matlab_model('Matlab_Models/Personalized/microbiota_model_samp_SRS011061.mat')
ref_mat = loadmat('Matlab_Models/Personalized/microbiota_model_samp_SRS011061.mat', simplify_cells=True)

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, d, fe, u
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, d, fe, u


In [ ]:
from scipy.sparse import issparse
import numpy as np

fields_model = model_data.keys()
fields_ref = ref_data.keys()
assert sorted(fields_model) == sorted(fields_ref), f"Field mismatch:\nModel: {sorted(fields_model)}\nRef: {sorted(fields_ref)}"
sorted(fields_model)

def compare_field(field):
    model_val = model_struct[field]
    ref_val = ref_struct[field]

    # Sparse matrix
    if issparse(model_val) and issparse(ref_val):
        if not (model_val != ref_val).nnz == 0:
            print(f"❌ Field '{field}' differs in sparse matrix values.")
        elif model_val.shape != ref_val.shape:
            print(f"❌ Field '{field}' shape mismatch: {model_val.shape} vs {ref_val.shape}")
        else:
            print(f"✅ Field '{field}' matches (sparse).")

    # Numeric array
    elif isinstance(model_val, np.ndarray) and np.issubdtype(model_val.dtype, np.number):
        try:
            if model_val.shape != ref_val.shape:
                print(f"❌ Field '{field}' shape mismatch: {model_val.shape} vs {ref_val.shape}")
                return
            if not np.allclose(model_val.astype(np.float64), ref_val.astype(np.float64), equal_nan=True):
                print(f"❌ Field '{field}' differs numerically.")
            else:
                print(f"✅ Field '{field}' matches (numeric).")
        except TypeError:
            print(f"❌ Field '{field}' failed numeric comparison due to invalid dtype.")
    
    # String/object array (rxns, mets, etc.)
    elif isinstance(model_val, np.ndarray) and model_val.dtype.kind in {'U', 'O'}:
        model_flat = sorted(_flatten_cell_array(model_val))
        ref_flat = sorted(_flatten_cell_array(ref_val))

        if model_flat != ref_flat:
            print(f"❌ Field '{field}' differs (sorted content mismatch)")
            for a, b in zip(model_flat, ref_flat):
                if a != b:
                    print(f"   ↳ {a} ≠ {b}")
                    break
        else:
            print(f"✅ Field '{field}' matches (sorted)")

    else:
        if np.array_equal(model_val, ref_val):
            print(f"✅ Field '{field}' matches (fallback).")
        else:
            print(f"❌ Field '{field}' differs (fallback).")

def _flatten_cell_array(arr):
    """Flatten nested array of strings or objects (cell array style)."""
    flat = []
    for item in arr.flatten():
        if isinstance(item, np.ndarray):
            flat.append(item.item() if item.size == 1 else str(item))
        else:
            flat.append(item)
    return flat

# Run comparison
for field in fields_model:
    compare_field(field)

In [ ]:
# # see what the coupling constraints are
model = cobra.io.load_matlab_model('Matlab_Models/Diet/microbiota_model_diet_SRS011061.mat')
mat = loadmat('Matlab_Models/Diet/microbiota_model_diet_SRS011061.mat', simplify_cells=True)

data = mat['model']
mat_C = data['C']
mat_d = data['d']
mat_dsense = data['dsense']
mat_ctrs = data['ctrs']

C = csr_matrix(mat_C.tocsr()) if not isinstance(mat_C, csr_matrix) else mat_C
rxn_ids = [r.id for r in model.reactions]

for i in range(10):
    ctr_name = mat_ctrs[i]
    row = C.getrow(i)
    nz_cols = row.nonzero()[1]
    coeffs = row.data
    rxns = [rxn_ids[j] for j in nz_cols]
    coeff_str = ', '.join([f"{coef}*{rxn}" for coef, rxn in zip(coeffs, rxns)])
    rhs = mat_d[i][0] if mat_d.ndim == 2 else mat_d[i]
    sense = mat_dsense[i]
    print(f"{ctr_name}: {coeff_str} {sense} {rhs}")

In [3]:
data = mat['model']
mat_C = data['C']
mat_d = data['d']
mat_dsense = data['dsense']
mat_ctrs = data['ctrs']

rdata = ref_mat['model']
rmat_C = rdata['C']
rmat_d = rdata['d']
rmat_dsense = rdata['dsense']
rmat_ctrs = rdata['ctrs']

In [7]:
print(pd.Series(mat_dsense).value_counts())
print(pd.Series(rmat_dsense).value_counts())

np.array_equal(mat_dsense, rmat_dsense)

L    10663
G     4437
Name: count, dtype: int64
L    10663
G     4437
Name: count, dtype: int64


True

In [8]:
print(pd.Series(mat_d).value_counts())
print(pd.Series(rmat_d).value_counts())

np.array_equal(mat_d, rmat_d)

0.0    15100
Name: count, dtype: int64
0.0    15100
Name: count, dtype: int64


True

In [11]:
np.array_equal(mat_ctrs, rmat_ctrs)

True

In [12]:
C = csr_matrix(mat_C.tocsr()) if not isinstance(mat_C, csr_matrix) else mat_C
rxn_ids = [r.id for r in model.reactions]

pc = []

for i in range(15100):
    row = C.getrow(i)
    nz_cols = row.nonzero()[1]
    coeffs = row.data
    rxns = [rxn_ids[j] for j in nz_cols]
    coeff_str = ', '.join([f"{coef}*{rxn}" for coef, rxn in zip(coeffs, rxns)])
    pc.append(coeff_str)

In [13]:
C = csr_matrix(rmat_C.tocsr()) if not isinstance(rmat_C, csr_matrix) else rmat_C
rxn_ids = [r.id for r in model.reactions]

rc = []

for i in range(15100):
    row = C.getrow(i)
    nz_cols = row.nonzero()[1]
    coeffs = row.data
    rxns = [rxn_ids[j] for j in nz_cols]
    coeff_str = ', '.join([f"{coef}*{rxn}" for coef, rxn in zip(coeffs, rxns)])
    rc.append(coeff_str)

In [16]:
np.array_equal(pc, rc)

True

In [ ]:
# check if reactions, metabolites, compartments, objectives equivalent between models

info_df = pd.DataFrame(columns=['Model', 'Reactions', 'Metabolites', 'Genes', 'Objective'])
models = [(samp1diet, "samp1"), (samp1pydiet, "samp1py"),]

# create a list to store the data for each model
model_data = []

for model in models:
  # add information from model to the list
  model_data.append({
      'Model': model[1],
      'Reactions': len(model[0].reactions),
      'Metabolites': len(model[0].metabolites),
      'Genes': len(model[0].genes),
      'Compartments': model[0].compartments,
      'Objective': model[0].objective.expression if model[0].objective else None
  })

# create the DataFrame from the list of model data
info_df = pd.DataFrame(model_data)
pd.set_option("display.max_colwidth", None)

info_df

In [ ]:
# if two models aren't showing equal reactions and metabolites:

matlab_model = samp1diet
python_model = samp1pydiet

# Compare Reactions

matlab_rxns = {rxn.id: rxn for rxn in matlab_model.reactions}
python_rxns = {rxn.id: rxn for rxn in python_model.reactions}

# Direct Comparison
common_rxns = set(matlab_rxns.keys()).intersection(python_rxns.keys())
only_in_matlab_rxns = set(matlab_rxns.keys()) - set(python_rxns.keys())
only_in_python_rxns = set(python_rxns.keys()) - set(matlab_rxns.keys())

# Display Number of Above
print(f"Total Num Reactions in MATLAB: {len(matlab_rxns)}")
print(f"Total Num Reactions in Python: {len(python_rxns)}\n")
print(f"Reactions in both MATLAB and Python models: {len(common_rxns)}")
print(f"Reactions only in MATLAB model: {len(only_in_matlab_rxns)}")
print(f"Reactions only in Python model: {len(only_in_python_rxns)}")
print(sorted([rxn for rxn in only_in_matlab_rxns]))
print(sorted([rxn for rxn in only_in_python_rxns]))

# Compare Mets

matlab_mets = {met.id: met for met in matlab_model.metabolites}
python_mets = {met.id: met for met in python_model.metabolites}

# Direct Comparison
common_mets = set(matlab_mets.keys()).intersection(python_mets.keys())
only_in_matlab_mets = set(matlab_mets.keys()) - set(python_mets.keys())
only_in_python_mets = set(python_mets.keys()) - set(matlab_mets.keys())

# Display Number of Above
print(f"Total Num mets in MATLAB: {len(matlab_mets)}")
print(f"Total Num mets in Python: {len(python_mets)}\n")
print(f"mets in both MATLAB and Python models: {len(common_mets)}")
print(f"mets only in MATLAB model: {len(only_in_matlab_mets)}")
print(f"mets only in Python model: {len(only_in_python_mets)}")

In [ ]:
matlab_models = [samp1diet, samp1diet, samp1diet, samp1diet]
python_models = [samp1pydiet, samp1pydiet, samp1pydiet, samp1pydiet]


# Asserting that reactions in the corr. models have the same reactants, products, coeffs, and reversability
# using a pandas df to display number of inaccuracies
# pd columns = model, num_reactant_diff, num_product_diff, num_coeff_diff, num_reversability_diff

df = pd.DataFrame(columns=['Matlab_Model', 'Python_model', 'Num_Reactant_Diff', 'Num_Product_Diff', 'Num_Coeff_Diff', 'Num_Reversability_Diff'])
num_reactant_diff = [0, 0, 0, 0]
num_product_diff = [0, 0, 0, 0]
num_coeff_diff = [0, 0, 0, 0]
num_reversability_diff = [0, 0, 0, 0]
num_bounds_diff = [0, 0, 0, 0]

for i in range(len(matlab_models)):
  matlab_model = matlab_models[i]
  python_model = python_models[i]

  for rxn in matlab_model.reactions:
      pyrxn = rxn.id.split('Diet_')[1] if 'Diet_' in rxn.id else rxn.id
      matlab_reactants = [met.id for met in rxn.reactants]
      python_reactants = [met.id for met in python_model.reactions.get_by_id(rxn.id).reactants]
      if set(matlab_reactants) != set(python_reactants):
        num_reactant_diff[i] += 1
        # print(rxn.id)
        # print(sorted(matlab_reactants))
        # print(sorted(python_reactants))

      matlab_products = sorted([met.id for met in rxn.products])
      python_products = sorted([met.id for met in python_model.reactions.get_by_id(rxn.id).products])
      if matlab_products != python_products:
        num_product_diff[i] += 1

      matlab_coeffs = [rxn.get_coefficient(met) for met in rxn.reactants + rxn.products]
      python_coeffs = [python_model.reactions.get_by_id(rxn.id).get_coefficient(met) for met in python_model.reactions.get_by_id(rxn.id).reactants + python_model.reactions.get_by_id(rxn.id).products]
      if sorted(matlab_coeffs) != sorted(python_coeffs):
        num_coeff_diff[i] += 1
        matlab_coeffs = [float(val) for val in matlab_coeffs]
        python_coeffs = [float(val) for val in python_coeffs]
        # print(rxn.id)
        # print(sorted(matlab_coeffs))
        # print(sorted(python_coeffs))

      if rxn.reversibility != python_model.reactions.get_by_id(rxn.id).reversibility:
        num_reversability_diff[i] += 1
        # print(rxn.id)
        # print(rxn.reversibility)
        # print(rxn.reversibility)
        
      if f'{rxn.lower_bound:.2f}' != f'{python_model.reactions.get_by_id(rxn.id).lower_bound:.2f}' or f'{rxn.upper_bound:.2f}' != f'{python_model.reactions.get_by_id(rxn.id).upper_bound:.2f}':
        num_bounds_diff[i] += 1
        print(rxn.id, rxn.bounds, python_model.reactions.get_by_id(rxn.id).bounds)

df['Matlab_Model'] = ['samp1diet', 'samp2diet', 'samp3diet', 'samp4diet']
df['Python_model'] = ['samp1pydiet', 'samp2py', 'samp3pydiet', 'samp4py']
df['Num_Rxns'] = [len(model.reactions) for model in matlab_models]
df['Num_Reactant_Diff'] = num_reactant_diff
df['Num_Product_Diff'] = num_product_diff
df['Num_Coeff_Diff'] = num_coeff_diff
df['Num_Reversability_Diff'] = num_reversability_diff
df['Num_Bounds_Diff'] = num_bounds_diff


# Easily check reaction bounds and reversability
data = []  # List to store data for DataFrame

for i in range(1):
    matlab_model = matlab_models[i]
    python_model = python_models[i]

    for rxn in matlab_model.reactions:
        if '[d]' in rxn.id and 'EX_' in rxn.id:
            data.append({
                'Matlab_Model': f'samp{i + 1}',  # Dynamically generate model name
                'Python_model': f'samp{i + 1}py',  # Dynamically generate model name
                'Reaction': rxn.id,
                'matlab_rev': rxn.reversibility,
                'python_rev': python_model.reactions.get_by_id(rxn.id).reversibility,
                'matlab_lower_bound': rxn.lower_bound,
                'python_lower_bound': python_model.reactions.get_by_id(rxn.id).lower_bound,
                'matlab_upper_bound': rxn.upper_bound,
                'python_upper_bound': python_model.reactions.get_by_id(rxn.id).upper_bound,
            })

rxndf = pd.DataFrame(data)  # Create DataFrame from collected data

In [ ]:
# Compare two models' optlangs
import re
from tqdm import tqdm

# Generate readable variables and constraints from optimized optlang containers

def gen_vars_constrs(model, opt_model):
    rxn_ids = [rxn.id for rxn in model.reactions]
    var_id_map = {f'v_{i}': rxn_ids[i] for i in range(len(rxn_ids))}

    vars = {}

    for var in opt_model.variables.keys():
        rxn_id = var_id_map.get(var)
        val = opt_model.variables[var]
        vars[rxn_id] = (val.lb, val.ub, val.primal)

    def replace_vars_in_str(expr_str, var_id_map):
        def replacer(match):
            var = match.group(0)
            return var_id_map.get(var, var)
        return re.sub(r'v_\d+', replacer, expr_str)

    constraints = []

    for cid, constraint in tqdm(opt_model.constraints.items()):
        expr_str = str(constraint.expression)
        renamed_str = replace_vars_in_str(expr_str, var_id_map)
        constraints.append(f"{cid}, {renamed_str}, {constraint.lb}, {constraint.ub}")

    return vars, constraints


# See if there are any differences in opt_model constraints, and identify sources
def build_constr_source_diff(pyconstraints, matconstraints):
    def normalize_expression(expr):
        expr = '+' + expr
        expr = expr.replace('- ', '-').replace('+- ', '-').replace('+-', '-').replace('+ ', '+')

        def round_match(match):
            num = float(match.group())
            return f"{round(num, 4)}"

        expr = re.sub(r'-?\d+\.\d+', round_match, expr)
        
        expr_list = expr.split(' ')
        return ' '.join(sorted(expr_list)) 

    def build_expr_map(constraints, source):
        expr_map = {}
        for s in constraints:
            expr = s.split(', ')[1]
            lb = s.split(', ')[2]
            ub = s.split(', ')[3]
            norm_expr = normalize_expression(expr)
            expr_map[norm_expr] = {'source': source, 'lb': lb, 'ub': ub}
        return expr_map
    
    py_map = build_expr_map(pyconstraints, "only_python")
    mat_map = build_expr_map(matconstraints, "only_matlab")

    # Merge both with preference for "both" if found in both
    records = []

    all_exprs = set(py_map.keys()).union(set(mat_map.keys()))

    for expr in all_exprs:
        if expr in py_map and expr in mat_map:
            records.append({
                'expression': expr,
                'source': 'both',
                'lb': (py_map[expr]['lb'], mat_map[expr]['lb']),
                'ub': (py_map[expr]['ub'], mat_map[expr]['ub'])
            })
        elif expr in py_map:
            records.append({
                'expression': expr,
                'source': 'only_python',
                'lb': py_map[expr]['lb'],
                'ub': py_map[expr]['ub']
            })
        else:
            records.append({
                'expression': expr,
                'source': 'only_matlab',
                'lb': mat_map[expr]['lb'],
                'ub': mat_map[expr]['ub']
            })

    common = len(set(py_map.keys()).intersection(set(mat_map.keys())))
    py = len(set(py_map.keys()) - set(mat_map.keys()))
    mat = len(set(py_map.keys()) - set(mat_map.keys()))

    print(common, py, mat)
    # Create the DataFrame
    df = pd.DataFrame(records)
    
    return df

def build_vars_flux_bounds_diff(pyvars, matvars):
    def get_value_differences(dict1, dict2):
        """
        Compares two dictionaries with the same keys and returns a new dictionary
        containing only the keys where the values differ, along with their values
        from both original dictionaries.
        """
        diff_dict = {}
        for key in dict1.keys():  # Assuming dict1 and dict2 have the same keys
            if dict1[key] != dict2[key]:
                diff_dict[key] = {'dict1_value': dict1[key], 'dict2_value': dict2[key]}
        return diff_dict

    differences = get_value_differences(pyvars, matvars)

    # Build row-by-row records from the differences
    diff_rows = []
    for rxn_id, val in differences.items():
        lb1, ub1, flux1 = val['dict1_value']
        lb2, ub2, flux2 = val['dict2_value']
        diff_rows.append({
            "Reaction ID": rxn_id,
            "Model1_LB": lb1,
            "Model1_UB": ub1,
            "Model1_Flux": flux1,
            "Model2_LB": lb2,
            "Model2_UB": ub2,
            "Model2_Flux": flux2,
            "Flux_Diff": abs(flux1 - flux2)
        })

    # Create and sort DataFrame
    df_diffs = pd.DataFrame(diff_rows)
    df_diffs = df_diffs.sort_values(by="Flux_Diff", ascending=False)

    return df_diffs



